In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional,SpatialDropout1D, GlobalMaxPooling1D, LayerNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd

###tokenization and sequencing:(run)

- tokenization: uses tf tokenizer to create a vocabulary

- sequence conversion: clean sequence -> sequence corresponding integer IDs, base on the vocabulary

- padding: pad all sequence with zeros until the reach maximum length, ensure all input stays the same